In [2]:
import re
import glob
import csv
from helpers import config
from helpers.loading import load_daily_data ,file_exist
from helpers.algorithm import find_best_delay
import pandas as pd
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:

pd.read_parquet('data/markets/NL/bbo/RDSa/2009-11-18-RDSa-bbo.parquet').tail()


,xltime,bid-price,bid-volume,ask-price,ask-volume
56762,40135.888653,62.60,1,65.62,2
56763,40135.888653,62.60,1,65.62,10
56764,40135.888757,62.59,2,65.62,10
56765,40135.889021,62.59,2,65.62,2
56766,40135.889945,62.59,2,0.00,0


In [26]:
get_all_dates()

['2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-12',
 '2009-03-13',
 '2009-03-16',
 '2009-03-17',
 '2009-03-18',
 '2009-03-19',
 '2009-03-20',
 '2009-03-23',
 '2009-03-24',
 '2009-03-25',
 '2009-03-26',
 '2009-03-27',
 '2009-03-30',
 '2009-03-31',
 '2009-04-28',
 '2009-04-29',
 '2009-04-30',
 '2009-05-01',
 '2009-05-04',
 '2009-05-05',
 '2009-05-06',
 '2009-05-07',
 '2009-05-08',
 '2009-05-11',
 '2009-05-12',
 '2009-05-13',
 '2009-05-14',
 '2009-05-15',
 '2009-05-18',
 '2009-05-19',
 '2009-05-20',
 '2009-05-21',
 '2009-05-26',
 '2009-05-27',
 '2009-05-28',
 '2009-05-29',
 '2009-06-01',
 '2009-06-02',
 '2009-06-03',
 '2009-06-04',
 '2009-06-05',
 '2009-06-08',
 '2009-06-09',
 '2009-06-10',
 '2009-06-11',
 '2009-06-12',
 '2009-06-15',
 '2009-06-16',
 '2009-06-17',
 '2009-06-18',
 '2009-06-19',
 '2009-06-22',
 '2009-06-23',
 '2009-06-24',
 '2009-06-25',
 '2009-06-26',
 '2009-06-29',
 '2009-06-30',
 '2009-07-

In [3]:
all_dates = get_all_dates(config["stock"],config["signal"])
print(f"{len(all_dates)} dates to process")

261 dates to process


In [4]:
fieldnames = ['date', 'market1','market2',"lag"]

results_path = config["files"]["results"]["all_best_lags"] # file where to write the computed lags

result_file_exists = file_exist(results_path) 
csvfile = open(results_path, 'a', newline='') 
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

if result_file_exists:
    processed_dates = set(pd.read_csv(results_path).date.unique())
else: 
    # if the file is new, we need to write headers
    writer.writeheader()
    processed_dates = set()


max_iterations = 10
start_time = time.time()
date_count = 0 # number of dates processed
for date_id,date in enumerate(all_dates):
    print(f"date:{date}, {date_id}:{len(all_dates)}, {100*date_id/len(all_dates):0.3f}%", end="\r")
    try:
        daily_data = load_daily_data(date)
    except:
        # in case all markets do not provide data for the given date, we skip the date
        continue
    
    # we skip the current date if it has already been processed
    if date in processed_dates:
        continue
        
    for i,n1 in enumerate(daily_data):
        for j,n2 in enumerate(daily_data):
            if i>j: # avoid symetric (corr(a,b)=corr(b,a)) and meaningless (corr(a,a)=1) calculations
                best_delay, delays, correlations, los, his = find_best_delay(daily_data,n1,n2,step_size=1000)
                # write the computed result
                writer.writerow({'date': date, 'market1': n1,'market2': n2,'lag': best_delay})
                writer.writerow({'date': date, 'market1': n2,'market2': n1,'lag': -best_delay})
    csvfile.flush() # flush every time we processed a date
    date_count+=1
    if date_count>=max_iterations:
        break
print()
print(f"{date_count} dates processed in {time.time()-start_time:0.2f}s")
csvfile.close()

data:2010-03-05, 45:261, 17.241%
10 dates processed in 297.89s
